# Introducción a los grafos

Un *grafo* es un conjunto de objetos llamados vértices (o nodos) que están unidos entre ellos por enlaces llamados aristas (líneas, o arcos). Los grafos pueden ser de diferentes tipos o tener distintas caracteristicas que los definen:

- Dirigos: Todos los aristas se dirigen de un vértice a otro.
- No dirigidos: Todos los aristas son bidireccionales (o no tienen una dirección definida).
- Ciclicos: Contienen al menos un grafo circular
- Aciclico: No contienen ciclos en los grafos (conocidos también como "arbol")
- Ponderado: Pesos/ponderaciones asignados en los vertices o aristas
- No ponderado: Todos los aristas y vertices no tienen peso.


Un ejemplo de grafo, son los DAGs representados en la planeación de ejecución de Spark.

### Qué problemas pueden representarse mediante grafos?
 😧

----

![Grafo](https://github.com/israelzuniga/dlatam-bigdata-workshop/blob/master/notebooks/img/grafo_acyc.png?raw=true)

El procesamiento de grafos involucra cruzar un grafo y ejecutar operaciones en cada nodo. Esto se enfoca en las relaciones de los elementos y la parelelización puede ser compleja. El primer desafio es extraer las relaciones entre elementos y desplegarlos en una estructura paralelizable. Una vez que se han identificado las relaciones, una forma de paralelizar es creando una lista de proximidad.

Una lista de proximidad representa las relaciones entre vértices en un grafo como una lista de aristas. Conceptualmente es más fácil iniciar con una matriz de proximidad, que representa las relaciones entre vértices un una matriz dispersa, y después crear una lista a partir de la matriz.

Tomando como ejemplo el anterior grafo ponderado. Se construye una matriz que consiste en cada vértice por cada arista. Si cada vértice está conectado directamente a otro, el valor de la intersección en la matriz es el peso asignado a la relación. Si no hay unión entre nodos, el valor de la intersección es cero (incluyendo la intersección del grafo consigo mismo).

Una matriz de proximidad quedaría así:

👽|A|B|C|D|E|F|G|H
--|--|--|--|--|--|--|--|--
A|0|**1**|**2**|**3**|0|0|0|0
B|0|0|0|0|**4**|0|0|0
C|0|0|0|0|**5**|**4**|0|0
D|0|0|0|0|0|0|**6**|0
E|0|0|0|0|0|**6**|0|0
F|0|0|0|0|0|0|0|**3**
G|0|0|0|0|0|**2**|0|0
H|0|0|0|0|0|0|0|0


La matriz de proximidad es ineficiente por lo dispersa que puede ser. Para convertirse en una lista de proximidad, se eliminan los ceros de la matriz y donde cada intersección de nodo y peso es representado en una tupla:

```
A, [(B,1), (C,2), (D,3)]
B, [(E,4)]
C, [(E,5), (F,4)]
D, [(G,6)]
E, [(F,6)]
F, [(H,3)]
G, [(F,2)]
```


Esta estructura puede comprimirse aún más en una lista de tripplets (trillizos), una estructura para definir un arista por su origen, destino y relación:

```
(A,B,1)
(A,C,2)
(A,D,3)
(B,E,4)
(C,E,5)
(C,F,4)
(D,G,6)
(E,F,6)
(F,H,3)
(G,F,2)
```

Esta nueva estructura representa los aristas en un grafo, sus vértices y los atributos para ejecutar las operaciones asociadas. Spark (incluyendo RDD, DataFrame API y otras APIs) puede ser usado para preparar los datos, después las extensiones para grafos pueden usarse para procesar los datos.


-----

GraphX: Spark's Graph Processing System

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages graphframes:graphframes:0.5.0-spark2.1-s_2.11,com.amazonaws:aws-java-sdk:1.10.34,org.apache.hadoop:hadoop-aws:2.6.0 pyspark-shell'
from pyspark import SparkContext, SparkConf
conf = (SparkConf()
        .setMaster('local[*]')
        .setAppName('My graphsx')
        .set('spark.executor.memory', '4g'))



sc = SparkContext(conf = conf)


from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)


from graphframes import *

In [ ]:
# Create a Vertex DataFrame with unique ID column "id"
v = sqlContext.createDataFrame([
  ("a", "Alice", 34),
  ("b", "Bob", 36),
  ("c", "Charlie", 30),
], ["id", "name", "age"])
# Create an Edge DataFrame with "src" and "dst" columns
e = sqlContext.createDataFrame([
  ("a", "b", "friend"),
  ("b", "c", "follow"),
  ("c", "b", "follow"),
], ["src", "dst", "relationship"])



In [ ]:
# Create GraphFrame
g = GraphFrame(v, e)


In [ ]:
# Task: Get in-degree of each vertex.? 
g.inDegrees.show()

# Task: Count the number of "follow" connections in the graph.
g.edges.filter(######).count()

# TaskL Run PageRank algorithm, and show results.
results = g.pageRank(resetProbability=0.01, maxIter=20)
results.vertices.select(#####).show()